# **Métodos de Ensemble — Módulo 3, Notebook 6/6**

---

## Índice

1. [Introdução ao Problema](#introducao)
2. [Bagging: Bootstrap Aggregating](#bagging)
   - [Intuição Prática](#bagging-intuicao)
   - [Formulação Formal](#bagging-formal)
   - [Implementação Manual](#bagging-manual)
3. [Boosting: AdaBoost](#boosting)
   - [Formulação Formal](#boosting-formal)
   - [Implementação com Sklearn](#boosting-sklearn)
4. [Stacking: Combinando Modelos Diversos](#stacking)
   - [Implementação com Sklearn](#stacking-sklearn)
   - [Desafio: Criar um Ensemble Híbrido](#stacking-desafio)
5. [Comparação: Bagging vs Boosting vs Stacking](#comparacao)
6. [Considerações Finais](#consideracoes)
7. [Resumo e Próximos Passos](#resumo)

---

<a id='introducao'></a>
## **Introdução ao Problema: Por Que Um Único Modelo Não É Suficiente?**

Vamos fazer um experimento. Imagine que você está construindo um modelo para prever se um cliente vai cancelar sua assinatura (churn).

**Cenário 1: Uma única Árvore de Decisão**

Você treina uma árvore de decisão e obtém 85% de acurácia no conjunto de teste. Ótimo, certo?

Mas espera... imagina que isso é o que acontece (e normalmente será) se treinar a mesma árvore com pequenas variações nos dados:

- **Tentativa 1**: 85% de acurácia
- **Tentativa 2**: 82% de acurácia (removemos 5% dos dados aleatoriamente)
- **Tentativa 3**: 88% de acurácia (diferentes dados de treino)
- **Tentativa 4**: 79% de acurácia
- **Tentativa 5**: 86% de acurácia

**Problemas:**

1. **Alta Variância**: Pequenas mudanças nos dados causam grandes variações no modelo
2. **Overfitting**: O modelo pode estar "decorando" os dados de treino
3. **Instabilidade**: Não podemos confiar plenamente em suas previsões

**A Solução dos Ensembles:**

Em vez de confiar em uma única árvore, será que faz sentido treinar várias árvores diferentes e combinar suas previsões?

- Se 8 de 10 árvores dizem "cliente vai cancelar", provavelmente ele vai...

<a id='bagging'></a>
## **Bagging: Bootstrap Aggregating**

**O que é Bagging?**

Bagging (Bootstrap Aggregating) é como pedir a opinião de várias pessoas que estudaram o mesmo assunto, mas com materiais ligeiramente diferentes.

**A Ideia:**

1. Pegamos nosso conjunto de dados original
2. Criamos várias "versões" diferentes dele (amostras bootstrap)
3. Treinamos um modelo em cada versão
4. Combinamos as previsões

**Bootstrap?**

Bootstrap é uma técnica de reamostragem com substituição. Imagine que você tem uma sacola com 100 bolas numeradas:

- Você retira uma bola, anota o número, e **devolve a bola** na sacola
- Repete isso 100 vezes
- Resultado: alguns números aparecem várias vezes, outros não aparecem

Isso cria uma versão "similar, mas diferente" do conjunto original! Retornaremos nisso

<a id='bagging-intuicao'></a>
### **Intuição Prática: Bagging com o Dataset Titanic**

Vamos usar o dataset do Titanic para entender o Bagging na prática.

**Mesmo Problema**: Prever se um passageiro sobreviveu ao naufrágio.

**Abordagem com uma única Árvore de Decisão:**
- Treina em todos os dados
- Pode fazer overfitting
- Instável com mudanças nos dados

**Abordagem com Bagging:**
- Cria 10 versões diferentes do dataset (bootstrap)
- Treina uma árvore em cada versão
- Cada árvore vota: "Sobreviveu" ou "Não sobreviveu"
- Decisão final: maioria dos votos

Vamos ver isso acontecer!

In [ ]:
# =======================================================
# IMPORTANDO AS BIBLIOTECAS
# =======================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Configuração de visualização (ignora, estou testando novas visualizações)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# =======================================================
# Carregando o Dataset do Titanic
# =======================================================

import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "yasserh/titanic-dataset",
    "Titanic-Dataset.csv",
    pandas_kwargs={"encoding": "latin", "usecols": [1, 2, 4, 5]}
)

# Pré-processamento
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Age'] = df['Age'].fillna(df['Age'].mean())

X = df[['Pclass', 'Sex', 'Age']].values
y = df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Tamanho do conjunto de treino: {len(X_train)}")
print(f"Tamanho do conjunto de teste: {len(X_test)}")

In [ ]:
# =======================================================
# Passo 1: Treinar uma ÚNICA Árvore de Decisão
# =======================================================

single_tree = DecisionTreeClassifier(random_state=42)
single_tree.fit(X_train, y_train)

y_pred_single = single_tree.predict(X_test)
metrics_single = classification_report(y_test, y_pred_single)

print(f"Métricas de uma única árvore:\n{metrics_single}")

# Vamos ver como essa árvore se comporta com diferentes amostras
print("\nTestando estabilidade (treinando em diferentes subconjuntos):")
accuracies = []

for i in range(10):
    # Criar uma amostra bootstrap
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_boot = X_train[indices]
    y_boot = y_train[indices]
    
    # Treinar árvore
    tree = DecisionTreeClassifier(random_state=i)
    tree.fit(X_boot, y_boot)
    
    # Avaliar
    acc = accuracy_score(y_test, tree.predict(X_test))
    accuracies.append(acc)
    print(f"Árvore {i+1}: {acc*100:.2f}%")

print(f"\nMédia: {np.mean(accuracies)*100:.2f}%")
print(f"Desvio padrão: {np.std(accuracies)*100:.2f}%")
print(f"\nInterpretação: A acurácia varia entre {min(accuracies)*100:.2f}% e {max(accuracies)*100:.2f}%")
print("nota: Cada árvore 'aprendeu' algo diferente.")

In [ ]:
# =======================================================
# Passo 2: Implementar Bagging Manualmente
# =======================================================
seeds = np.arange(10)

# Armazenar os modelos
models = []
predictions = []

print("Treinando ensemble com Bagging...")
for i in seeds:
    # 1. Criar amostra bootstrap
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_boot = X_train[indices]
    y_boot = y_train[indices]
    
    # 2. Treinar modelo na amostra bootstrap
    tree = DecisionTreeClassifier(random_state=i)
    tree.fit(X_boot, y_boot)
    models.append(tree)
    
    # 3. Fazer previsões
    pred = tree.predict(X_test)
    predictions.append(pred)
    
    print(f"Árvore {i+1} treinada em {len(np.unique(indices))} amostras únicas (de {len(X_train)} possíveis)")


predictions = np.array(predictions)

print(f"\nShape das previsões: {predictions.shape}")
print(f"({seeds.size} árvores, {len(X_test)} amostras de teste)")

# =======================================================
# Passo 3: Combinar previsões por votação majoritária
# =======================================================

# Para cada amostra de teste, contar os votos
final_predictions = []

for i in range(len(X_test)):
    votes = predictions[:, i]  # Votos de todas as árvores para a amostra i
    # Votação majoritária
    prediction = np.bincount(votes).argmax()
    final_predictions.append(prediction)

final_predictions = np.array(final_predictions)

# Avaliar
metrics_bagging = classification_report(y_test, final_predictions)

print(f"\n{'='*50}")
print(f"RESULTADOS:")
print(f"{'='*50}")
print(f"Métricas com Bagging ({seeds.size} árvores):\n{metrics_bagging}")
metric = accuracy_score(y_test, final_predictions)
print(f"Acurácia com Bagging ({seeds.size} árvores): {metric*100:.2f}%")
print(f"Melhoria: {(metric - accuracy_score(y_test, y_pred_single))*100:.2f} pontos percentuais")

In [ ]:
# =======================================================
# Visualização: Como cada árvore vota
# =======================================================

# Vamos ver as previsões para as primeiras 10 amostras de teste
n_samples_to_show = 10

fig, ax = plt.subplots(figsize=(14, 6))

# Criar matriz de votos
vote_matrix = predictions[:, :n_samples_to_show].T

# Plotar heatmap
im = ax.imshow(vote_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

# Configurar eixos
ax.set_xlabel('Árvore', fontsize=12)
ax.set_ylabel('Amostra de Teste', fontsize=12)
ax.set_title('Votação de Cada Árvore (0 = Não Sobreviveu, 1 = Sobreviveu)', fontsize=14, fontweight='bold')
ax.set_xticks(range(seeds.size))
ax.set_xticklabels([f'Árvore {i+1}' for i in range(seeds.size)], rotation=45)
ax.set_yticks(range(n_samples_to_show))
ax.set_yticklabels([f'Amostra {i+1}' for i in range(n_samples_to_show)])
ax.grid(False)

# Adicionar valores nas células
for i in range(n_samples_to_show):
    for j in range(seeds.size):
        text = ax.text(j, i, int(vote_matrix[i, j]), 
                      ha="center", va="center", color="black", fontsize=10)


# Adicionar informação sobre a votação final
for i in range(n_samples_to_show):
    votes = vote_matrix[i]
    final_vote = "Sobreviveu" if np.sum(votes) > seeds.size/2 else "Não Sobreviveu"
    vote_count = int(np.sum(votes))
    ax.text(seeds.size, i, f'{final_vote}\n({vote_count}/{seeds.size})', 
           ha="left", va="center", fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nInterpretação:")
print("- Verde: Árvore previu 'Sobreviveu' (1)")
print("- Vermelho: Árvore previu 'Não Sobreviveu' (0)")
print("- Decisão final: maioria dos votos")

<a id='bagging-formal'></a>
### **Formulação Formal do Bagging**

**Entrada:**

1. Conjunto de treinamento: $D = \{(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), ..., (x^{(n)}, y^{(n)})\}$

2. Algoritmo de aprendizado base $\mathcal{A}$ (e.g., Árvore de Decisão)

3. Número de modelos no ensemble: $T$

**Processamento:**

Para cada modelo $t = 1, 2, ..., T$:

1. **Criar amostra bootstrap** $D_t$:
   - Amostrar $n$ exemplos de $D$ com substituição
   - Aproximadamente 63.2% das amostras originais aparecem em $D_t$
   - As amostras restantes (~36.8%) são chamadas de "out-of-bag" (OOB), (por quê?)

2. **Treinar modelo base**:
   - $h_t = \mathcal{A}(D_t)$
   - Cada modelo $h_t$ é treinado independentemente

**Saída - Predição:**

Para uma nova amostra $x^o$:

- votação majoritária:

$$\hat{y} = \text{moda}\{h_1(x^o), h_2(x^o), ..., h_T(x^o)\}$$

**Propriedades Importantes:**

1. **Redução de Variância**: 
   - Se os modelos base têm variância $\sigma^2$, a variância do ensemble é aproximadamente $\frac{\sigma^2}{T}$ (assumindo independência)
   - Na prática, $\frac{\sigma^2}{T}$ nunca acontece porque há correlação entre modelos, mas de fato a variância é reduzida

2. **Out-of-Bag Error (OOB)**:
   - Para cada amostra de treino, podemos usar os modelos que NÃO a viram no treino para estimar o erro
   - Isso fornece uma estimativa de validação "gratuita" sem precisar de um conjunto de validação separado
   e.g

   Suponha $n=5$ amostras e $T=3$ modelos:

   - Modelo 1 treinou com amostras: {1, 2, 3, 5} → amostra 4 está OOB
   - Modelo 2 treinou com amostras: {1, 2, 4, 5} → amostra 3 está OOB  
   - Modelo 3 treinou com amostras: {2, 3, 4, 5} → amostra 1 está OOB

   Para avaliar a amostra 4:
   - Use apenas Modelo 1 (que não viu a amostra 4)
   - $\hat{y}^{OOB}_4 = h_1(x^{(4)})$

   Para avaliar a amostra 1:
   - Use apenas Modelo 3 (que não viu a amostra 1)
   - $\hat{y}^{OOB}_1 = h_3(x^{(1)})$


3. **Bias-Variance Tradeoff**:
   - O grande dilema!
   - Por isso, funciona melhor com modelos de baixo bias e alta variância (como árvores mais profundas)

<a id='bagging-manual'></a>
### **Desafio: Implementar Bagging do Zero**

Esse é um pouco mais complicado, mas tente implementar uma classe `BaggingClassifier` que funcione com qualquer classificador base.

**Requisitos:**

1. Deve aceitar qualquer classificador como base (DecisionTree, KNN, etc.)
2. Criar amostras bootstrap
3. Treinar múltiplos modelos
4. Combinar previsões por votação
5. Calcular OOB error (desafio extra, se pra ti estiver facil)

In [ ]:
from sklearn.base import clone

class BaggingClassifier:
    def __init__(self, base_estimator, n_estimators=10, random_state=None):
        """
        Inicializa o BaggingClassifier
        
        Parâmetros:
        -----------
        base_estimator: classificador base (e.g., DecisionTreeClassifier())
        n_estimators: número de modelos no ensemble
        random_state: seed para reprodutibilidade
        """
        pass
    
    def _create_bootstrap_sample(self, X, y):
        """
        Cria uma amostra bootstrap de X e y
        
        Retorna:
        --------
        X_boot, y_boot: amostra bootstrap
        oob_indices: índices das amostras out-of-bag
        """
        pass
    
    def fit(self, X, y):
        """
        Treina o ensemble
        
        Processo:
        1. Para cada estimador:
           a. Criar amostra bootstrap
           b. Clonar o estimador base
           c. Treinar o clone na amostra bootstrap
           d. Armazenar o modelo e os índices OOB
        """
        pass
    
    def predict(self, X):
        """
        Faz previsões usando votação majoritária
        
        Retorna:
        --------
        y_pred: array de previsões
        """
        pass
    
    def score(self, X, y):
        """
        Calcula a acurácia do modelo
        """
        pass
    
    def oob_score(self):
        """
        DESAFIO EXTRA: Calcular o OOB error
        
        Para cada amostra de treino:
        - Usar apenas os modelos que NÃO a viram no treino
        - Fazer votação com esses modelos
        - Comparar com o label verdadeiro
        
        Retorna:
        --------
        oob_accuracy: acurácia calculada nas amostras OOB
        """
        pass

<a id='boosting'></a>
## **Boosting: AdaBoost (Adaptive Boosting)**

**O que é Boosting?**

Boosting é muito diferente de Bagging. Em vez de treinar modelos independentemente, **cada modelo foca nos erros do modelo anterior**.

Imagine um professor ajudando um aluno:
- Sessão 1: Ensina os conceitos básicos
- Sessão 2: Foca nos tópicos que o aluno errou
- Sessão 3: Foca mais nos erros anteriores
- ...E assim por diante, até que o aluno dominar

**A Ideia do AdaBoost:**

1. Treinamos um modelo fraco no dataset
2. Aumentamos o peso das amostras que ele errou
3. Treinamos outro modelo fraco com ênfase nos erros anteriores
4. Repetimos e combinamos com votação ponderada

**Modelo Fraco?**

Um "modelo fraco" (weak learner) é um modelo que tem acurácia ligeiramente melhor que 50% (melhor que chute aleatório).

- **Exemplo**: Uma árvore de decisão com profundidade 1 (decision stump)
- **Não precisa** ser complexo, ao contrário! Modelos simples funcionam melhor com boosting

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# =======================================================
# 1. BaggingClassifier com Árvores de Decisão
# =======================================================

bagging_tree = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
    random_state=42,
    oob_score=True  # Calcular OOB score
)

bagging_tree.fit(X_train, y_train)

print("="*60)
print("BAGGING COM ÁRVORES DE DECISÃO")
print("="*60)
print(f"Acurácia no teste: {bagging_tree.score(X_test, y_test)*100:.2f}%")
print(f"OOB Score: {bagging_tree.oob_score_*100:.2f}%")
print(f"Número de estimadores: {bagging_tree.n_estimators}")

# =======================================================
# 2. Random Forest (Bagging + Feature Randomness)
# =======================================================

# Random Forest é um tipo especial de Bagging que:
# - Usa árvores de decisão como base
# - Adiciona aleatoriedade na seleção de features

random_forest = RandomForestClassifier(
    n_estimators=50,
    random_state=42,
    oob_score=True
)

random_forest.fit(X_train, y_train)

print("\n" + "="*60)
print("RANDOM FOREST (Bagging + Feature Randomness)")
print("="*60)
print(f"Acurácia no teste: {random_forest.score(X_test, y_test)*100:.2f}%")
print(f"OOB Score: {random_forest.oob_score_*100:.2f}%")

# Feature importance
feature_names = ['Pclass', 'Sex', 'Age']
importances = random_forest.feature_importances_
print("\nImportância das features:")
for name, importance in zip(feature_names, importances):
    print(f"  {name}: {importance*100:.2f}%")

# =======================================================
# 3. Bagging com outros classificadores
# =======================================================

print("\n" + "="*60)
print("BAGGING COM DIFERENTES CLASSIFICADORES BASE")
print("="*60)

# KNN
bagging_knn = BaggingClassifier(
    estimator=KNeighborsClassifier(n_neighbors=5),
    n_estimators=50,
    random_state=42
)
bagging_knn.fit(X_train, y_train)
print(f"\nBagging + KNN: {bagging_knn.score(X_test, y_test)*100:.2f}%")

# Naive Bayes
bagging_nb = BaggingClassifier(
    estimator=GaussianNB(),
    n_estimators=50,
    random_state=42
)
bagging_nb.fit(X_train, y_train)
print(f"Bagging + Naive Bayes: {bagging_nb.score(X_test, y_test)*100:.2f}%")

In [ ]:
# =======================================================
# Parâmetros Importantes do BaggingClassifier
# =======================================================

# n_estimators: número de modelos base
# max_samples: número/proporção de amostras para cada bootstrap
# max_features: número/proporção de features para cada modelo
# bootstrap: se True, usa bootstrap; se False, usa todo dataset
# oob_score: se True, calcula OOB score

configs = [
    {'n_estimators': 10, 'max_samples': 1.0, 'max_features': 1.0},
    {'n_estimators': 50, 'max_samples': 0.8, 'max_features': 1.0},
    {'n_estimators': 100, 'max_samples': 1.0, 'max_features': 0.8},
    {'n_estimators': 50, 'max_samples': 0.7, 'max_features': 0.7},
]

print("Testando diferentes configurações:\n")
for i, config in enumerate(configs, 1):
    bag = BaggingClassifier(
        estimator=DecisionTreeClassifier(),
        random_state=42
    )
    bag.fit(X_train, y_train)
    
    print(f"{i}. n_estimators={config['n_estimators']}, "
          f"max_samples={config['max_samples']}, "
          f"max_features={config['max_features']}")
    print(f"   Acurácia: {bag.score(X_test, y_test)*100:.2f}%")

-Adicione o OOB score e veja o que acontece- Como você faria uma validação cruzada?

**Dica: Random Forest vs Bagging**

- **BaggingClassifier com árvores**: Cada árvore vê TODAS as features em cada divisão
- **RandomForest**: Cada divisão da árvore vê apenas um subconjunto aleatório de features

Por que isso importa?
- Random Forest cria árvores mais **decorrelacionadas**
- Resultado: geralmente melhor performance que Bagging puro
- Trade-off: um pouco mais de bias, mas muito menos variância

## **Boosting: Aprendizado Sequencial**

**O que é Boosting?**

- **Bagging**: Modelos treinados em **paralelo**, independentemente
  - "Todos vocês, estudem o material e me digam o que acham"

- **Boosting**: Modelos treinados em **sequência**, cada um focando nos erros do anterior
  - "Você 1, estude tudo. Você 2, foque no que o 1 errou. Você 3, foque no que os anteriores erraram..."

**A ideia:**

1. Treinar um modelo "fraco" (weak learner)
2. Identificar em quais amostras ele errou
3. Dar mais **peso/atenção** a essas amostras difíceis
4. Treinar um novo modelo focado nelas
5. Repetir o processo
6. Combinar todos os modelos, dando mais peso aos melhores

### **Intuição do AdaBoost (Adaptive Boosting)**

AdaBoost foi um dos primeiros e mais populares algoritmos de boosting. Vamos entender como funciona!

**O Processo:**

1. **Início**: Todas as amostras têm peso igual (e.g., 1/n cada)

2. **Treinar modelo 1**: 
   - Treina em todas as amostras
   - Identifica quais errou

3. **Ajustar pesos**:
   - ↑ Aumenta peso das amostras que errou
   - ↓ Diminui peso das que acertou

4. **Treinar modelo 2**:
   - Agora "presta mais atenção" nas amostras difíceis
   - Tem que acertar as que o modelo 1 errou

5. **Repetir** o processo T vezes

6. **Combinar**: Cada modelo tem um peso baseado em sua acurácia
   - Modelos melhores têm mais influência na decisão final

Vamos ver isso na prática!

In [ ]:
# =======================================================
# Simulação Visual do AdaBoost

# ideia:
# - Mostrar como os pesos das amostras mudam a cada iteração
# - Mostrar como os erros são tratados
# - Mostrar a importância (alpha) de cada modelo 
# * Não se preocupe com o cálculo exato ainda
# =======================================================

# Dataset 2D simples
np.random.seed(42)
n_samples = 40 
X_viz = np.random.randn(n_samples, 2)  
y_viz = (X_viz[:, 0] + X_viz[:, 1] > 0).astype(int)
y_viz = 2 * y_viz - 1  # Converter para {-1, +1}

# Inicializar
weights = np.ones(n_samples) / n_samples
n_iterations = 3
models = []
alphas = []
all_weights = []

# Treinar AdaBoost
for t in range(n_iterations):
    all_weights.append(weights.copy())
    
    # Treinar weak learner
    weak_learner = DecisionTreeClassifier(max_depth=1, random_state=t)
    weak_learner.fit(X_viz, y_viz, sample_weight=weights)
    predictions = weak_learner.predict(X_viz)
    
    # Calcular erro e alpha
    incorrect = predictions != y_viz
    error = np.sum(weights * incorrect) / np.sum(weights)
    alpha = 0.5 * np.log((1 - error) / (error))
    
    # Atualizar pesos
    weights = weights * np.exp(-alpha * y_viz * predictions)
    weights = weights / np.sum(weights)
    
    models.append(weak_learner)
    alphas.append(alpha)

# Visualização com 3 iterações
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for t in range(n_iterations):
    ax = axes[t]
    weights_t = all_weights[t]
    predictions = models[t].predict(X_viz)
    incorrect = predictions != y_viz

    sizes = 500 * weights_t / weights_t.max()
    
    # corretos
    correct = ~incorrect
    ax.scatter(X_viz[correct, 0], X_viz[correct, 1], 
              c=y_viz[correct], s=sizes[correct], 
              alpha=0.7, cmap='seismic', edgecolors='k', linewidth=2)
    
    # errados
    if incorrect.sum() > 0:
        ax.scatter(X_viz[incorrect, 0], X_viz[incorrect, 1], 
                  s=sizes[incorrect]*1.5, marker='X', 
                  c=predictions[incorrect],
                  cmap='seismic', edgecolors='r', linewidth=1, zorder=5, alpha=0.9)
    
    # Título
    ax.set_title(f'Modelo {t+1}\n(Alpha = {alphas[t]:.2f})', 
                fontsize=12, fontweight='bold')
    ax.grid(alpha=0.1)
    ax.set_xlabel('Feature 1', fontsize=12)
    if t == 0:
        ax.set_ylabel('Feature 2', fontsize=12)

plt.suptitle('AdaBoost\n(Tamanho = Peso)', 
            fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("\nINTERPRETAÇÃO:")
print("=" * 60)
print("Pontos MAIORES = amostras com MAIS peso")
print("X = erros -> cor = classe prevista")
print("  - X azul = modelo previu -1 (mas era +1)")
print("  - X vermelho = modelo previu +1 (mas era -1)")
print("A cada iteração, o modelo foca nos erros anteriores")
print(f"\nQuantidade de erros por modelo: {[int((models[t].predict(X_viz) != y_viz).sum()) for t in range(n_iterations)]}")
print(f"\nImportância dos modelos (alphas): {[f'{a:.2f}' for a in alphas]}")


<a id='boosting-formal'></a>
### **Formulação Formal do AdaBoost**

**Entrada:**

1. Conjunto de treinamento: $D = \{(x^{(1)}, y^{(1)}), ..., (x^{(n)}, y^{(n)})\}$ onde $y^{(i)} \in \{-1, +1\}$

2. Algoritmo de aprendizado fraco $\mathcal{A}$

3. Número de iterações: $T$

**Processamento:**

**Inicialização:**
- Pesos uniformes: $w^{(1)}_i = \frac{1}{n}$ para $i = 1, ..., n$

**Para cada iteração** $t = 1, 2, ..., T$:

1. **Treinar modelo fraco**:
   - $h_t = \mathcal{A}(D, w^{(t)})$
   - Cada amostra $(x^{(i)}, y^{(i)})$ tem peso $w^{(t)}_i$

2. **Calcular erro ponderado**:
   
   $$\epsilon_t = \sum_{i=1}^{n} w^{(t)}_i \cdot \mathbb{I}(h_t(x^{(i)}) \neq y^{(i)})$$
   
   onde $\mathbb{I}$ é a função indicadora (1 se erro, 0 se acerto)

3. **Calcular peso do modelo** (influência na predição final):
   
   $$\alpha_t = \frac{1}{2}\ln\left(\frac{1-\epsilon_t}{\epsilon_t}\right)$$
   
   - Se $\epsilon_t$ é pequeno (modelo bom) → $\alpha_t$ é grande
   - Se $\epsilon_t \approx 0.5$ (chute aleatório) → $\alpha_t \approx 0$

4. **Atualizar pesos das amostras**:
   
   $$w^{(t+1)}_i = w^{(t)}_i \cdot \exp(-\alpha_t \cdot y^{(i)} \cdot h_t(x^{(i)}))$$
   
   Simplificando:
   - Se $h_t$ acerta $(y^{(i)} = h_t(x^{(i)}) \Rightarrow y^{(i)}\cdot h_t(x^{(i)}) = 1)$: peso diminui (multiplicado por $e^{-\alpha_t}$)
   - Se $h_t$ erra $(y^{(i)} \neq h_t(x^{(i)})\Rightarrow y^{(i)}\cdot h_t(x^{(i)}) = -1)$: peso aumenta (multiplicado por $e^{\alpha_t}$)

5. **Normalizar pesos**:
   
   $$w^{(t+1)}_i = \frac{w^{(t+1)}_i}{\sum_{j=1}^{n}w^{(t+1)}_j}$$

**Saída - Predição:**

Para uma nova amostra $x^o$:

$$H(x^o) = \text{sign}\left(\sum_{t=1}^{T}\alpha_t \cdot h_t(x^o)\right)$$

- Votação ponderada: modelos melhores ($\alpha_t$ maior) têm mais influência
- $\text{sign}()$ retorna +1 ou -1 (a classe final)

<a id='boosting-sklearn'></a>
### **Implementação com Sklearn: AdaBoost e Gradient Boosting**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

# =======================================================
# 1. AdaBoost
# =======================================================

ada_boost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),  # Stumps
    n_estimators=50,
    learning_rate=1.0,
    random_state=42
)

ada_boost.fit(X_train, y_train)

print("="*60)
print("ADABOOST")
print("="*60)
print(f"Acurácia no teste: {ada_boost.score(X_test, y_test)*100:.2f}%")

# Pesos dos estimadores
print(f"\nPrimeiros 5 pesos dos modelos (alpha):")
print(ada_boost.estimator_weights_[:5])

# =======================================================
# 2. Gradient Boosting
# =======================================================

# Gradient Boosting é uma generalização do AdaBoost
# Em vez de ajustar pesos, ele ajusta os resíduos (erros) diretamente

gradient_boost = GradientBoostingClassifier(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

gradient_boost.fit(X_train, y_train)

print("\n" + "="*60)
print("GRADIENT BOOSTING")
print("="*60)
print(f"Acurácia no teste: {gradient_boost.score(X_test, y_test)*100:.2f}%")

# Feature importance
feature_names = ['Pclass', 'Sex', 'Age']
importances = gradient_boost.feature_importances_
print("\nImportância das features:")
for name, importance in zip(feature_names, importances):
    print(f"  {name}: {importance*100:.2f}%")

# =======================================================
# 3. Comparação: Bagging vs Boosting
# =======================================================

from sklearn.ensemble import RandomForestClassifier

models = {
    'Single Tree': DecisionTreeClassifier(random_state=42),
    'Bagging (Random Forest)': RandomForestClassifier(n_estimators=50, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42)
}

print("\n" + "="*60)
print("COMPARAÇÃO DE MODELOS")
print("="*60)

for name, model in models.items():
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    print(f"{name:25s}: {acc*100:.2f}%")

In [ ]:
# =======================================================
# Parâmetros Importantes do Gradient Boosting
# =======================================================

# n_estimators: número de árvores
# learning_rate: taxa de aprendizado (shrinkage)
#   - Valores menores = mais árvores necessárias, mas melhor generalização
# max_depth: profundidade máxima de cada árvore
# subsample: fração de amostras usadas para treinar cada árvore
#   - < 1.0 = Stochastic Gradient Boosting (reduz overfitting)

configs = [
    {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 1.0},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1.0},
    {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 1.0},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 0.8},
]

print("Testando diferentes configurações de Gradient Boosting:\n")
for i, config in enumerate(configs, 1):
    gb = GradientBoostingClassifier(**config, random_state=42)
    gb.fit(X_train, y_train)
    
    print(f"{i}. n_est={config['n_estimators']}, lr={config['learning_rate']}, "
          f"depth={config['max_depth']}, subsample={config['subsample']}")
    print(f"   Acurácia: {gb.score(X_test, y_test)*100:.2f}%\n")

**Dica: Learning Rate no Boosting**

O `learning_rate` (ou shrinkage) controla o quanto cada árvore contribui para a predição final.

- **learning_rate = 1.0**: Cada árvore contribui totalmente
  -  Converge rápido
  - Porém, Mais propenso a overfitting

- **learning_rate = 0.1**: Cada árvore contribui apenas 10%
  - Melhor generalização
  - Porém, Precisa de mais árvores (maior n_estimators)

**Regra:** learning_rate baixo + mais árvores = melhor performance (mas mais lento)

<a id='stacking'></a>
## **Stacking: Combinando Diferentes Modelos**

**O que é Stacking?**

Stacking (Stacked Generalization) é como além de uma **equipe de especialistas diferentes** ter um **coordenador** que aprende a melhor forma de combinar suas opiniões.

**A Ideia:**

1. **Nível 0 (Base)**: Treinar vários modelos diferentes
   - KNN, Árvore de Decisão, Naive Bayes, SVM, etc.
   - Cada um tem suas próprias forças e fraquezas

2. **Gerar meta-features**: As previsões desses modelos base se tornam novas features

3. **Nível 1 (Meta)**: Treinar um "meta-modelo" que aprende a combinar as previsões dos modelos base
   - Geralmente um modelo simples (Regressão Logística)
   - Aprende qual modelo confiar em cada situação

**Diferença de Bagging/Boosting:**

- **Bagging/Boosting**: Múltiplos modelos do MESMO tipo (várias árvores)
- **Stacking**: Modelos DIFERENTES + meta-modelo que aprende a combiná-los

### **Intuição do Stacking**

Imagine que você está decidindo se um e-mail é spam:

**Modelos Base (Nível 0):**

1. **Naive Bayes** diz: "90% spam" 
2. **KNN** diz: "60% spam"
3. **Árvore de Decisão** diz: "80% spam" 
4. **SVM** diz: "85% spam" 
**Meta-Modelo (Nível 1):**

Em vez de simplesmente fazer a média (78.75%), o meta-modelo aprende que:

- "Quando Naive Bayes está confiante (>85%), confie nele"
- "Quando os modelos discordam muito, prefira o SVM (por quê?)"
- "Se KNN está muito diferente dos outros, ignore-o"

Resultado final: **92% spam**

Vamos entender na prática:

In [ ]:
# =======================================================
# Implementação Manual do Stacking
# =======================================================

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

# Passo 1: Definir modelos base
base_models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42)
}

# Passo 2: Treinar modelos base e obter previsões
print("="*60)
print("NÍVEL 0: Treinando modelos base")
print("="*60)

# Armazenar previsões dos modelos base
base_predictions_train = []
base_predictions_test = []

for name, model in base_models.items():
    # Treinar no conjunto de treino
    model.fit(X_train, y_train)
    
    # Fazer previsões (probabilidades)
    # Usamos cross-validation para evitar overfitting no meta-modelo
    train_pred = cross_val_predict(
        model, X_train, y_train, cv=5, method='predict_proba'
    )[:, 1]  # Probabilidade da classe 1
    
    # Previsões no teste
    test_pred = model.predict_proba(X_test)[:, 1]
    
    base_predictions_train.append(train_pred)
    base_predictions_test.append(test_pred)
    
    acc = model.score(X_test, y_test)
    print(f"{name:20s}: {acc*100:.2f}%")

# Converter para arrays numpy
X_meta_train = np.column_stack(base_predictions_train)
X_meta_test = np.column_stack(base_predictions_test)

print(f"\nShape das meta-features:")
print(f"  Treino: {X_meta_train.shape}")
print(f"  Teste: {X_meta_test.shape}")

# Passo 3: Treinar meta-modelo
print("\n" + "="*60)
print("NÍVEL 1: Treinando meta-modelo")
print("="*60)

meta_model = LogisticRegression(random_state=42)
meta_model.fit(X_meta_train, y_train)

# Fazer previsões finais
y_pred_stacking = meta_model.predict(X_meta_test)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)

print(f"Acurácia do Stacking: {accuracy_stacking*100:.2f}%")

# Visualizar os coeficientes do meta-modelo
print("\nCoeficientes do meta-modelo (importância de cada modelo base):")
for name, coef in zip(base_models.keys(), meta_model.coef_[0]):
    print(f"  {name:20s}: {coef:.4f}")

**DESAFIO: Como seria a visualização para o Stacking?**

### **Formulação Formal do Stacking**

**Entrada:**

1. Conjunto de treinamento: $D = \{(x^{(1)}, y^{(1)}), ..., (x^{(n)}, y^{(n)})\}$

2. Conjunto de $K$ algoritmos de nível 0 (base): $\{\mathcal{A}_1, \mathcal{A}_2, ..., \mathcal{A}_K\}$

3. Algoritmo de nível 1 (meta): $\mathcal{A}_{meta}$

**Processamento:**

**Nível 0 - Treinar Modelos Base:**

Para cada algoritmo $\mathcal{A}_k$ ($k = 1, ..., K$):

1. **Treinar modelo**: $h_k = \mathcal{A}_k(D)$

2. **Gerar meta-features** usando validação cruzada para evitar overfitting:
   - Dividir $D$ em $V$ folds
   - Para cada fold $v$:
     - Treinar $h_k$ nos outros $V-1$ folds
     - Prever no fold $v$
   - Resultado: $\tilde{h}_k(x^{(i)})$ para todo $i$ (previsões out-of-fold)

**Criar Dataset de Meta-Features:**

Para cada amostra $(x^{(i)}, y^{(i)})$:

$$x_{meta}^{(i)} = [\tilde{h}_1(x^{(i)}), \tilde{h}_2(x^{(i)}), ..., \tilde{h}_K(x^{(i)})]$$

O novo dataset meta é: $D_{meta} = \{(x_{meta}^{(1)}, y^{(1)}), ..., (x_{meta}^{(n)}, y^{(n)})\}$

**Nível 1 - Treinar Meta-Modelo:**

$$h_{meta} = \mathcal{A}_{meta}(D_{meta})$$

**Saída - Predição:**

Para uma nova amostra $x^o$:

1. **Obter previsões dos modelos base**:
   
   $$x_{meta}^o = [h_1(x^o), h_2(x^o), ..., h_K(x^o)]$$

2. **Aplicar meta-modelo**:
   
   $$\hat{y} = h_{meta}(x_{meta}^o)$$

**Variações Importantes:**

1. **Features Originais no Meta-Modelo**: 
   - Em vez de usar apenas as previsões dos modelos base, podemos concatenar com as features originais:
   
   $$x_{meta}^{(i)} = [x^{(i)}, \tilde{h}_1(x^{(i)}), ..., \tilde{h}_K(x^{(i)})]$$

2. **Multi-Level Stacking**:
   - Podemos ter múltiplos níveis: Nível 0 → Nível 1 → Nível 2 → ...
   - Cada nível usa as previsões do nível anterior como features

<a id='stacking-sklearn'></a>
### **Implementação com Sklearn: StackingClassifier**

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

# =======================================================
# 1. Stacking Básico
# =======================================================

# Definir modelos base
estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('tree', DecisionTreeClassifier(max_depth=5, random_state=42)),
    ('nb', GaussianNB()),
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42))
]

# Criar stacking com Regressão Logística como meta-modelo
stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5  # Validação cruzada para gerar meta-features
)

stacking.fit(X_train, y_train)

print("="*60)
print("STACKING CLASSIFIER (sklearn)")
print("="*60)
print(f"Acurácia: {stacking.score(X_test, y_test)*100:.2f}%")

# =======================================================
# 2. Stacking com passthrough (features originais)
# =======================================================

# passthrough=True adiciona as features originais ao meta-modelo
stacking_passthrough = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5,
    passthrough=True  # Incluir features originais
)

stacking_passthrough.fit(X_train, y_train)

print("\n" + "="*60)
print("STACKING COM PASSTHROUGH")
print("="*60)
print(f"Acurácia: {stacking_passthrough.score(X_test, y_test)*100:.2f}%")

# =======================================================
# 3. Diferentes meta-modelos
# =======================================================

meta_models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

print("\n" + "="*60)
print("TESTANDO DIFERENTES META-MODELOS")
print("="*60)

for name, meta_model in meta_models.items():
    stack = StackingClassifier(
        estimators=estimators,
        final_estimator=meta_model,
        cv=5
    )
    stack.fit(X_train, y_train)
    acc = stack.score(X_test, y_test)
    print(f"{name:25s}: {acc*100:.2f}%")

**Dica: Escolhendo Modelos Base para Stacking**

Para um bom Stacking, escolha modelos **diversos**:

**Bons para Stacking:**
- Modelos com princípios diferentes (KNN + Árvore + Naive Bayes)
- Modelos lineares + não-lineares
- Modelos que cometem erros diferentes

**Ruins para Stacking:**
- Múltiplas variações do mesmo modelo (3 árvores de decisão diferentes)
- Modelos muito correlacionados
- Modelos que sempre concordam

*Se os modelos base sempre preveem a mesma coisa, o stacking não ajuda*

<a id='stacking-desafio'></a>
### **Desafio: Criar um Ensemble Híbrido**

Agora é sua vez! Combine Bagging, Boosting e Stacking em um único ensemble.

**Objetivo:** Criar um StackingClassifier onde os modelos base sejam ensembles!

**Estrutura sugerida:**

Nível 0 (Modelos Base):

In [ ]:
# =======================================================
# SEU CÓDIGO AQUI!
# =======================================================

# Dica: Use StackingClassifier com estimators que sejam
# BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, etc.

# Exemplo de estrutura:
# hybrid_ensemble = StackingClassifier(
#     estimators=[
#         ('rf', RandomForestClassifier(...)),
#         ('gb', GradientBoostingClassifier(...)),
#         ('ada', AdaBoostClassifier(...)),
#         ('bag_svm', BaggingClassifier(SVC(...), ...))
#     ],
#     final_estimator=...
# )

pass

<a id='comparacao'></a>
## **Comparação: Bagging vs Boosting vs Stacking**

**Como escolher qual usar?**

Agora que estudamos os três principais métodos de ensemble, vamos comparar suas características e entender quando usar cada um.

In [ ]:
# =======================================================
# Idéia:
# - Comparamos todos os modelos (single tree, bagging, boosting, stacking)
# - Medimos tempo de treino, acurácia no teste, cross-validation score
# - Analisamos a importância de cada modelo no ensemble

# =======================================================

import time
from sklearn.model_selection import cross_val_score

# Definir todos os modelos para comparação
models_comparison = {
    # Baseline
    'Decision Tree (baseline)': DecisionTreeClassifier(random_state=42),
    
    # Bagging
    'Bagging (Decision Tree)': BaggingClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=50,
        random_state=42
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=50,
        random_state=42
    ),
    
    # Boosting
    'AdaBoost': AdaBoostClassifier(
        n_estimators=50,
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=50,
        random_state=42
    ),
    
    # Stacking
    'Stacking': StackingClassifier(
        estimators=[
            ('knn', KNeighborsClassifier(n_neighbors=5)),
            ('tree', DecisionTreeClassifier(max_depth=5, random_state=42)),
            ('nb', GaussianNB()),
            ('rf', RandomForestClassifier(n_estimators=30, random_state=42))
        ],
        final_estimator=LogisticRegression(),
        cv=5
    )
}

# Avaliar todos os modelos
results = []

print("="*80)
print("COMPARAÇÃO COMPLETA DE ENSEMBLE METHODS")
print("="*80)
print(f"{'Modelo':<30} {'Acurácia Teste':<15} {'CV Score (mean)':<20} {'Tempo (s)':<10}")
print("-"*80)

for name, model in models_comparison.items():
    # Medir tempo de treino
    start_time = time.time()
    
    # Treinar
    model.fit(X_train, y_train)
    
    # Acurácia no teste
    test_acc = model.score(X_test, y_test)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    # Tempo
    elapsed_time = time.time() - start_time
    
    results.append({
        'name': name,
        'test_acc': test_acc,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'time': elapsed_time
    })
    
    print(f"{name:<30} {test_acc*100:>6.2f}%        {cv_mean*100:>6.2f}% ± {cv_std*100:>4.2f}%    {elapsed_time:>6.2f}s")

print("-"*80)

<a id='consideracoes'></a>
## **Considerações Finais**

### **Quando Usar Cada Método?**

**Bagging:**
-  Quando você quer um modelo **simples e robusto**
-  Random Forest é geralmente a melhor escolha
-  Reduz variância sem aumentar muito a complexidade
-  Pode ser **paralelizado** → rápido em multicore
-  **Exemplo**: Classificação de imagens, regressão

**Boosting:**
-  Quando você precisa de **máxima acurácia** (geralmente o melhor desempenho)
-  Quando você tem modelos com **alto bias** (ex: stumps)
-  Quando você pode **ajustar cuidadosamente os hiperparâmetros**
-  Cuidado com overfitting em datasets pequenos
-  **Exemplo**: Competições de ML (Kaggle), ranking de busca

**Stacking:**
-  Quando você quer **combinar modelos fundamentalmente diferentes**
-  Quando você já tem **vários modelos bons** e quer espremer mais performance
-  Em **competições** onde cada 0.1% de acurácia importa
-  Mais complexo de implementar e manter
-  **Exemplo**: Sistemas de recomendação, previsão de séries temporais

### **Considerações Práticas**

**1. Interpretabilidade:**

Todos os métodos de ensemble reduzem a interpretabilidade:
- Um modelo único é mais fácil de explicar
- Ensembles são "caixas-pretas"
- Use **feature importance** para ter alguma interpretação

**2. Hiperparâmetros Críticos:**

**Bagging:**
- `n_estimators`: Quantos modelos (mais = melhor, mas com retorno decrescente)
- `max_samples`: Tamanho das amostras bootstrap
- `max_features`: Features aleatórias (para Random Forest)

**Boosting:**
- `n_estimators`: Número de modelos (cuidado com overfitting)
- `learning_rate`: Taxa de aprendizado (menor = mais modelos necessários)
- `max_depth`: Profundidade das árvores (geralmente shallow)

**Stacking:**
- Escolha dos **modelos base** (diversidade é chave!)
- Escolha do **meta-modelo** (geralmente simples: LogisticRegression)
- `cv`: Número de folds (importante para evitar overfitting)

**3. Dicas Práticas**

**Comece simples**: Random Forest é geralmente uma ótima escolha inicial
- Robusto, fácil de usar, bom desempenho out-of-the-box

**Para máxima performance**: Gradient Boosting (ou XGBoost/LightGBM)
- Mas requer mais tuning de hiperparâmetros

**Compute vs Accuracy**:
- Bagging: Pode ser paralelizado → rápido
- Boosting: Sequencial → mais lento
- Stacking: Overhead adicional → mais lento ainda

**Evite ensembles de ensembles (em produção)**:
- Stacking de Boosting pode dar overfitting
- Complexidade de manutenção explode
- Trade-off entre 0.5% de acurácia vs simplicidade

**4. Aplicações no Mundo Real**

1. **Detecção de Spam**: Random Forest ou Gradient Boosting
2. **Diagnóstico Médico**: Stacking 
3. **Previsão de Churn**: Gradient Boosting (maximiza acurácia)
4. **Sistemas de Recomendação**: Stacking (combine collaborative filtering + content-based)
5. **Detecção de Fraude**: Random Forest (lidar com desbalanceamento)

**5. Bibliotecas para Aprofundamento**

- **[XGBoost](https://xgboost.readthedocs.io/en/stable/)**: Gradient Boosting extremamente otimizado
- **[LightGBM](https://lightgbm.readthedocs.io/en/stable/Python-Intro.html)**: Gradient Boosting ainda mais rápido (da Microsoft)
- **[CatBoost](https://catboost.ai/docs/en/concepts/python-installation)**: Gradient Boosting para features categóricas (da Yandex)
- **[VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)**: Ensemble simples por votação (sklearn)

Todos são variações/melhorias dos métodos que estudamos.

---

<a id='resumo'></a>
## **Resumo e Próximos Passos**

### **Aprendizados Principais**

1. **Ensembles reduzem variância** combinando múltiplos modelos
2. **Bagging** treina modelos independentemente em amostras bootstrap
3. **Boosting** treina modelos sequencialmente, focando em erros anteriores
4. **Stacking** usa um meta-modelo para aprender a combinar previsões
5. **Random Forest** é geralmente a melhor escolha para começar
6. **Gradient Boosting** oferece máxima acurácia com tuning adequado

### **Próximos Tópicos**

- Técnicas avançadas: XGBoost, LightGBM, CatBoost
- Tuning de hiperparâmetros com Grid/Random Search
- Interpretabilidade em modelos de ensemble (SHAP, etc.)
- Ensembles em problemas de séries temporais
- Ensembles para regressão vs classificação

<-- [**Anterior: Árvores de Decisão**](05_arvores_classificacao.ipynb) | [**Próximo: Módulo 04 — Regressão**](../04_Regressao/01_regressao_linear_multipla.ipynb) -->